This notebook is used for preprocessing the scrapied data from steam, like remove special characters as \n, \t and emojis.

In [10]:
import psycopg2
import re
import emoji
from collections import defaultdict

In [ ]:
# establish a connection to database
postgres_conn_param = {
    'host': 'localhost',
    'port': '5432',
    'user': 'postgres',
    'password': '12345678',
    'database': 'steam'
}

conn = psycopg2.connect(**postgres_conn_param)
cursor = conn.cursor()

In [3]:
cursor.execute("SELECT review_text FROM game_reviews ORDER BY random() LIMIT 100000;")
data = cursor.fetchall()

In [4]:
special_chars = defaultdict(int)

In [5]:
def find_special_chars(text):
    for char in text:
        if (not char.isalnum()) and (not char.isspace()):
            special_chars[char] += 1

In [7]:
for (text,) in data:
    find_special_chars(text)

In [8]:
cursor.close()
conn.close()

In [9]:
for char, count in special_chars.items():
    print(f"'{char}': {count}")

'*': 4247
',': 142227
'.': 220528
'!': 37772
'😭': 24
'(': 12897
''': 70033
')': 14713
'♥': 25554
'£': 129
'-': 43510
':': 13170
'’': 2316
'$': 1558
'%': 2179
'"': 12158
'/': 14869
'[': 550
']': 554
'+': 3570
'•': 185
'?': 8792
'☑': 2232
'☐': 8945
';': 2503
'<': 653
'\': 356
'&': 1110
'–': 166
'€': 129
'~': 760
'✅': 141
'🔲': 885
'⣿': 42511
'⠟': 574
'⠋': 896
'⠁': 1489
'⠀': 63101
'⠉': 2336
'⠻': 708
'⢺': 75
'⠆': 91
'⠜': 67
'⠿': 1273
'⠛': 1797
'⣀': 2084
'⣠': 857
'⣤': 1476
'⣶': 1179
'⣦': 634
'⡄': 707
'⢀': 1866
'⢠': 483
'⣾': 1196
'⢿': 1252
'⡧': 56
'⠇': 348
'⣮': 144
'⣭': 192
'⡻': 111
'⣽': 371
'⣒': 50
'⣜': 82
'⠐': 221
'⢐': 166
'⠢': 167
'⢰': 384
'⢸': 1640
'⣏': 90
'⡟': 597
'⠂': 322
'⢈': 105
'⣷': 1292
'⣞': 101
'⣸': 313
'⡿': 1386
'⠗': 122
'⠈': 1554
'⢻': 650
'⠑': 242
'⢘': 139
'⢹': 378
'⡇': 1352
'⠴': 193
'⠶': 316
'⢽': 121
'⠣': 103
'⠹': 221
'⣧': 801
'⡀': 1347
'⣴': 901
'⡙': 105
'⠙': 970
'⠄': 5204
'⡃': 50
'⢷': 166
'⠾': 84
'⢡': 54
'=': 2735
'>': 1047
'{': 1697
'}': 1699
'‘': 302
'@': 1654
'´': 262
'◡': 7

In [18]:
def clean_text(text):
    #replace emojis with their names such as ⭐ for :star, and 😸 for :cat
    text = emoji.demojize(text, delimiters=(":", ":"))
    #replace _ with spaces making emoji names easier for tokenlization
    text = text.replace('_', ' ')
    #new line characters and tabs
    text = re.sub(r'[\n\t]', ' ', text)
    #replace consuctive spaces to single space
    text = re.sub(r'\s+', ' ', text)
    return text

In [19]:
clean_text('I love this game ♥♥♥!! \n you must play it')

'I love this game :heart suit::heart suit::heart suit:!! you must play it'